# RUDI Node tools: *rudinode-get* library

This library offers tools to take advantage of the [external API](https://app.swaggerhub.com/apis/OlivierMartineau/RUDI-PRODUCER) of a RUDI Producer node (also referred as RUDI node).

File [rudi_node_getter.py](lib_rudi_meta/rudi_node_getter.py) contains a class `RudiNodeGetter` that makes it easier 
to access a RUDI Producer node data.
This class fetches all the metadata once and let you access the resulting metadata.

## Initializing the `RudiNodeGetter` object
This object only takes the RUDI node URL as a parameter.
You can optionaly give some identifier that will be used for every request made to the node.
> RUDI node external API does not need any identification. But you can give an indication in the request header.

In [ ]:
from lib_rudi_meta.rudi_node_getter import RudiNodeGetter
from utils.log import log_d

rudi_node_url = 'https://bacasable.fenix.rudi-univ-rennes1.fr'
rudi_node_info = RudiNodeGetter(server_url=rudi_node_url, headers_user_agent='RudiNodeGetExample01')

## Access to metadata information
The `RudiNodeGetter` object lets you access and take advantage of the metadata stored on the node:
- access to the full list of metadata

In [ ]:
log_d('RudiNode info', 'metadata nb', rudi_node_info.metadata_count)
log_d('RudiNode info', 'metadata list', rudi_node_info.metadata_list)
log_d('RudiNode info', 'metadata 1', rudi_node_info.metadata_list[0])

- access to the producers and contacts information

In [ ]:
log_d('RudiNode info', 'list of producers', rudi_node_info.organization_list)
log_d('RudiNode info', 'producer names', rudi_node_info.organization_names)

log_d('RudiNode info', 'list of contacts', rudi_node_info.contact_list)
log_d('RudiNode info', 'contact names', rudi_node_info.contact_names)

- access to the classification tags

log_d('RudiNode info', 'themes', rudi_node_info.themes)
log_d('RudiNode info', 'keywords', rudi_node_info.keywords)

## Filtering the metadata
`RudiNodeGetter` object offers some tools to filter the metadata with a partial JSON.
> You will need to understand how a RUDI metadata is structured to create adequate filters. See RUDI node [external API](https://app.swaggerhub.com/apis/OlivierMartineau/RUDI-PRODUCER) documentation for this.
> All the elements given in the filter will need to be matched in the metadata for it to be kept in the end result of the filtering operation.

In [ ]:
example_filter = {'producer': {'organization_id': '1d6bc543-07ed-46f6-a813-958edb73d5f0', 'organization_name': 'SIB (Test)'}}
log_d('Metadata filter', 'filter metadata', rudi_node_info.filter_metadata(example_filter))

Some shortcuts have been implemented to make it easier to filter the metadata:

In [ ]:
prod_name = 'SIB (Test)'
log_d('Metadata filter', 'metadata from producer', rudi_node_info.get_metadata_with_producer(prod_name))
cont_name = 'Bacasable'
log_d('Metadata filter', f"metadata with contact '{cont_name}'", rudi_node_info.get_metadata_with_contact(
    cont_name))

thm = 'citizenship'
log_d('Metadata filter', 'metadata with theme', rudi_node_info.get_metadata_with_theme(thm))
kw = ['répartition', 'Commune']
log_d('Metadata filter', f"metadata with keywords '{kw}'", rudi_node_info.get_metadata_with_keyword(kw))

meta_id = 'f48b4bcd-bba3-47ba-86e6-c0754b748728'
log_d('Metadata filter', f"metadata with id '{meta_id}'", rudi_node_info.get_metadata_with_id(meta_id))

## Downloading a file
`RudiNodeGetter` object also provides a method to download the data stored on the node:

In [ ]:
log_d('Download files', f"download_media_for_metadata '{meta_id}'", rudi_node_info.download_media_for_metadata(metadata_id=meta_id, local_download_dir='../1-dwnld'))